In [1]:
using TensorFlow
using CSV: read
using Distributions: Normal
using ProgressMeter: @showprogress

In [2]:
# Preprocessing the data
f = CSV.read("./gyroscope.csv")

T = length(f[:,11])
# T = 24382
Nw = 3                       # Window size
Nc = 3                       # number of inputs
Ncw = Nc * Nw                # input size
TNw = Int64(floor(T/Nw))

dim1 =  Int64(floor(Ncw / 2))
dim2 =  Int64(floor(dim1 / 2))

raw_train = Array{Float32, 2}(Nc,T)
raw_train = f[:,11:13]

s = Array{Float32, 2}(Ncw,TNw)
for i=1:TNw
    s[:,i] .= vec(convert(Array,f[Nw*(i-1)+1:Nw*i,11:13]))
end
s

9×8127 Array{Float32,2}:
 -0.0082558   -0.0066579   -0.00292948  …  -0.00798948  -0.00878843
 -0.00532632  -0.00585895  -0.00292948     -0.00852212  -0.0111853 
 -0.00346211  -0.00452737  -0.00692422     -0.00878843  -0.0111853 
  0.0215716    0.0149137    0.0218379       0.0218379    0.0218379 
  0.0205063    0.0143811    0.0186421       0.021039     0.0221042 
  0.0191748    0.0191748    0.0125169   …   0.0215716    0.0229032 
 -0.00186421  -0.00239685  -0.00692422     -0.00452737  -0.00479369
 -0.0015979   -0.0015979   -0.00905475     -0.00372843  -0.00292948
 -0.00292948  -0.00532632  -0.00878843     -0.00559264  -0.00213053

In [3]:
function create_tensor(input)
    output = nn.sigmoid(W1t * nn.sigmoid(W2t * nn.sigmoid(W2 * nn.sigmoid(W1 * input + b1) + b2) + b3) + b4)
    return output
end

create_tensor (generic function with 1 method)

In [4]:
function weight_variable(shape)
    initial = map(Float32, rand(Normal(0, .01), shape...))
    return Variable(initial)
end

function bias_variable(shape)
    initial = fill(Float32(.1), shape...)
    return Variable(initial)
end

sess = Session(Graph())

@tf begin
    
    x = placeholder(Float32, shape=[Ncw, 1])
        
    W1 = weight_variable([dim1, Ncw])
    b1 = bias_variable([dim1, 1])
    enc1 = nn.sigmoid(W1 * x + b1)
                
    W2 = weight_variable([dim2, dim1])
    b2 = bias_variable([dim2, 1])
    enc2 = nn.sigmoid(W2 * enc1 + b2)
    
    # this is a tied weight
    W2t = transpose(W2)
    b3 = bias_variable([dim1, 1])
    dec1 = nn.sigmoid(W2t * enc2 + b3)

    # this is a tied weight too
    W1t = transpose(W1)
    b4 = bias_variable([Ncw, 1])
    dec2 = nn.sigmoid(W1t * dec1 + b4)
    
end

2018-07-06 22:24:51.314722: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA


<Tensor dec2:1 shape=unknown dtype=Float32>

In [5]:
losses = Array{Any, 1}()
input = Array{Any, 2}(Ncw,1)


@tf begin
    data = constant(s)
    
    for i=1:5
        # input is the i-th column of the data
        input = slice(data,[1,i],[Ncw,1])
        push!(losses, reduce_mean((create_tensor(input) - input)^2 ))
    end
    
    # TODO: add regularization term
    loss = reduce_sum(stack(losses))
    optimizer = train.minimize(train.AdamOptimizer(1e-4), loss)
end

run(sess, global_variables_initializer())

In [6]:
@showprogress for i=1:10
    cur_loss, _ = run(sess, (loss, optimizer))
    println(cur_loss)
end

1.3663452


Progress:  10%|████                                     |  ETA: 0:00:05

1.3660104
1.3656445
1.3651774
1.3647976
1.3644048
1.3640414
1.3636181
1.3632135
1.3628091


Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
